In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Aim

Scripts to compute and analyse the R-factor of the RUSLE-equation. The
R-factor is a measure for the total erosivity of a number of rainfall events
within a defined timeframe (year, month, number of days). The factor is
computed by calculating the yearly sum of -for every rainfall event- the sum
of the depth of rainfall (mm) and the kinetic energy, and taking the mean
over all years. For the formula's, we refer to the [CN-WS Pascal model documentation](https://docs.fluves.net/cnws-pascal/watem-sedem.html#rusle-factors)

## Imports
Main imports

In [17]:
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
from pathlib import Path
import sys;
import os

Package imports

In [18]:
from rfactor.process import (ErosivityData, compute_rainfall_statistics, compute_rainfall_statistics)
from rfactor.rfactor import compute_rfactor

Working directory

In [19]:
cwd = Path().resolve()

**Set folders**  
The input files are defined by text files (extension: .txt) that hold non-zero rainfall timeseries. The data are split per station and per year with a specific datafile tag:  

KMI_6414_2004.txt  
KMI_6414_2005.txt  
...  
KMI_6434_2003.txt  
KMI_6434_2004.txt  
...  

In [20]:
fmap_rainfall=  Path(r"../../tests/data/test_rainfalldata")
fmap_erosivity = Path(r"../../src/rfactor/results")
fmap_results = Path.cwd() / "results"

In [21]:
fmap_results

WindowsPath('c:/users/sachagobeyn/github/rfactor/src/rfactor/results')

**Rainfall files summary file**  
An overview of the present datafiles for the analysis is saved in a  `files.csv` file (example in *./tests/data*). This file can be used to remove specific files from the analysis (column `consider`):

   | source        | datafile      | consider  |
  | ------------- |:-------------:| ---------:|
  | KMI	          | KMI_6414_2004 | 0         |
  | KMI	          | KMI_6414_2005 | 1         |
  | KMI	          | KMI_6414_2006 | 1         |
  | ...           | ...           | ...       |


In [22]:
txt_files=  Path(r"data\belgium\files.csv")
txt_files.exists()

False

**Run Model**  
The current implemenation makes use of a Matlab engine, which requires Matlab to be installed. Future versions of this package will use Python. Results are 
written to the *./src/rfactor/results*-folder.

In [26]:
compute_rfactor(fmap_rainfall,fmap_results,"matlab")

> c:\users\sachagobeyn\github\rfactor\src\rfactor\rfactor.py(35)compute_rfactor()
     33         ]
     34         import pdb;pdb.set_trace()
---> 35         check_call(cmd)
     36     else:
     37         rfactor_python(rainfall_inputdata_folder)



ipdb>  cmd


['matlab', '-nodesktop', '-r', "main('('C:\\\\Users\\\\SachaGobeyn\\\\GitHub\\\\rfactor\\\\tests\\\\data\\\\test_rainfalldata', 'c:\\\\users\\\\sachagobeyn\\\\github\\\\rfactor\\\\src\\\\rfactor\\\\results')');exit;"]


ipdb>  f"main('{rainfall_inputdata_folder.resolve(),str(results_folder)}')


*** SyntaxError: EOL while scanning string literal


...

"main('('C:\\\\Users\\\\SachaGobeyn\\\\GitHub\\\\rfactor\\\\tests\\\\data\\\\test_rainfalldata', 'c:\\\\users\\\\sachagobeyn\\\\github\\\\rfactor\\\\src\\\\rfactor\\\\results')');exit;"


ipdb>  f"main('{str(rainfall_inputdata_folder.resolve())},{str(results_folder)}');exit;"


"main('C:\\Users\\SachaGobeyn\\GitHub\\rfactor\\tests\\data\\test_rainfalldata,c:\\users\\sachagobeyn\\github\\rfactor\\src\\rfactor\\results');exit;"


ipdb>  exit


BdbQuit: 

**Set-up analysis**  
Create a erosivitydata object, build the data set with the *files.csv* file and load the rainfall and EI30 data.  
__NOTE__: 
1. Set the director back to the location of this file
2. If the <ins>**Matlab**</ins> is used, make sure **to first let the simulation end before executing the next line!!!!!**

In [ ]:
os.chdir(cwd)
data = ErosivityData(fmap_rainfall,fmap_erosivity)
df_files = data.build_data_set(txt_files)
data.load_data(df_files)
df_files.to_csv("overview_files_stations.csv")

**Rainfall statistics**  
Compute rainfall statistics

In [ ]:
compute_rainfall_statistics(data.load_rainfall(),pd.read_csv("data/belgium/stations.csv"))

## Ukkel  
Compute R-value for specific years for Ukkel (KMI_6447 and KMI_F3)

In [ ]:
timeseries = [range(1898,2003,1),
              range(2003,2021,1),
              range(1898,2021,1),
              range(1996,2021,1),
              range(1991,2021,1),
              range(1990,2001,1),
              range(2000,2021,1)]
for i in timeseries:
    print(i)
    df_R=data.load_R(["KMI_6447","KMI_FS3"], i)
    print(np.mean(df_R["value"]))

In [ ]:
len(data.stations)

## Flanders and Belgium
**All stations expect Ukkel**

In [ ]:
stations_belgium_excl_ukkel = [station for station in data.stations if station not in ["KMI_6447","KMI_FS3"]]
df_R=data.load_R(stations_belgium_excl_ukkel)
print(np.mean(df_R["value"]))
len(stations_belgium_excl_ukkel)

**All stations except Ukkel and Wallonia**

In [ ]:
stations_flanders_excl_ukkel = [station for station in data.stations if station not in  ["KMI_6447","KMI_FS3","KMI_6455","KMI_6459","KMI_6472","KMI_6494","KMI_6484"]]
df_R=data.load_R(stations_flanders_excl_ukkel, i)
print(np.mean(df_R["value"]))
len(stations_flanders_excl_ukkel)

**Plot the distribution of all R-values for Belgium, excluding the data from Ukkel**

In [ ]:
stations_flanders_excl_ukkel = [station for station in data.stations if station not in  ["KMI_6447","KMI_FS3"]]
df_R=data.load_R(stations_flanders_excl_ukkel, i)
print(np.mean(df_R["value"]))
len(stations_flanders_excl_ukkel)
plt.hist(df_R["value"],20,color=[0.8]*3,label=r"Jaarlijkse waarden voor alle 55 stations in België")
plt.plot([1239,1239],[0,120],color=[0.2]*3,ls=":",label="Ukkel (30-jaar referentie periode)")
plt.ylabel(r"#")
plt.xlabel(r"R-value [MJ mm ha$^{-1}$ h$^{-1}$ jaar$^{-1}$]")
plt.ylim([0,130])
plt.legend()

**All stations of the VMM (Flanders)**

In [ ]:
stations_flanders_excl_ukkel = [station for station in data.stations if "KMI" not in station]
df_R=data.load_R(stations_flanders_excl_ukkel, i)
print(np.mean(df_R["value"]))
len(stations_flanders_excl_ukkel)

**Compute values per year over all stations**

In [ ]:
df_R.groupby("station").aggregate({"value":[np.mean,np.std],"year":lambda x:len(x)}).sort_values(('year', '<lambda>'),ascending=False).reset_index()

## Monthly analysis
Get the EI30-values for 2018 based on two Ukkel station ("KMI_6447","KMI_FS3")  

In [ ]:
df_EI30 = data.load_EI30(["KMI_6447","KMI_FS3"],range(1898,2021,1))

**Data formatting**

In [ ]:
df_EI30["m"] = df_EI30.index.strftime("%m%d")
df_EI30.loc[df_EI30["m"]=="0229","m"] = "0228"
df_m = df_EI30.groupby("m").aggregate({"value":[np.mean,np.std,lambda x:np.percentile(x,25),lambda x:np.percentile(x,75)]})
df_m["mean"] = df_m[("value","mean")];df_m["std"] = df_m[("value","std")];df_m["p25"] = df_m[('value', '<lambda_0>')];df_m["p75"] = df_m[('value', '<lambda_1>')];df_m["p25_e"] = df_m["mean"]-df_m['p25'];df_m["p75_e"] = df_m['p75']-df_m["mean"]

**Plot**

In [ ]:
fig = plt.figure(figsize=(8,4))
x = np.arange(len(df_m))
y_gv = [26,20,24,27,70,77,144,102,68,50,37,32]
plt.bar(x-0.225,df_m["mean"],yerr=df_m[["p25_e","p75_e"]].T.values,color=[0.80]*3,width=0.45,label="Ukkel (1898-2020))")
plt.bar(x+0.225,y_gv,width=0.45,color=[0.5]*3,label="Verstraeten $\it{et. al}$ (2001)")
plt.ylabel("R-value")
ax = fig.axes
plt.xticks(x,["Januari","Februari","Maart","April","Mei","Juni","Juli","Augustus","September","Oktober","November","December"],rotation=45)
plt.legend(loc=2,facecolor ="white")
plt.ylabel(r"R-waarde [MJ mm ha$^{-1}$ h$^{-1}$ maand$^{-1}$]")